<a href="https://colab.research.google.com/github/jimmyliao/llm-workshop/blob/main/JimmyLiao_Breeze_7B_FC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- Remember to use A100 Runtime
- Original Notebook: https://huggingface.co/MediaTek-Research/Breeze-7B-FC-v1_0
- Modified by: Jimmy Liao <sjliao@gmail.com>

In [1]:
!nvidia-smi

Fri Sep 27 15:46:44 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0              44W / 350W |      2MiB / 40960MiB |      0%      Default |
|                                         |                      |             Disabled |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -U vllm==0.4.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 22.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.1/37.1 MB 49.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
!git clone https://github.com/mtkresearch/mtkresearch.git
%cd mtkresearch
!pip install .
!pip show mtkresearch

Cloning into 'mtkresearch'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (48/48), done.
remote: Total 58 (delta 20), reused 30 (delta 8), pack-reused 0 (from 0)
Receiving objects: 100% (58/58), 20.91 KiB | 20.91 MiB/s, done.
Resolving deltas: 100% (20/20), done.
/content/mtkresearch
Processing /content/mtkresearch
  Preparing metadata (setup.py) ... done
  Created wheel for mtkresearch: filename=mtkresearch-0.0.6-py3-none-any.whl size=6142 sha256=6333c3ea6fb3766c72f561930bae277a784a2aac8bd974fc2ac88bb3c4a59ca3
  Stored in directory: /tmp/pip-ephem-wheel-cache-wdfq43r4/wheels/12/82/1a/965f8030402f7056ec00e919cb17fc03c9ff2c52a22f925ac3
Successfully built mtkresearch
Name: mtkresearch
Version: 0.0.6
Summary: mtkresearch
Home-page: 
Author: 
Author-email: 
License: Apache License 2.0
Location: /usr/local/lib/python3.10/dist-packages
Requires: 
Required-by: 


In [5]:
# !nvidia-smi --query-gpu=name --format=csv,noheader | wc -l
# !nvidia-smi --query-gpu=name --format=csv,noheader

In [4]:
import torch
free_gpu_memory, total_gpu_memory = torch.cuda.mem_get_info()
print(f"Free GPU memory: {free_gpu_memory / 1024 ** 2:.2f} MB")
print(f"Total GPU memory: {total_gpu_memory / 1024 ** 2:.2f} MB")

Free GPU memory: 40090.81 MB
Total GPU memory: 40513.81 MB


### Hosting by VLLM

In [5]:
from vllm import LLM, SamplingParams

llm = LLM(
    model='MediaTek-Research/Breeze-7B-FC-v1_0',
    # tensor_parallel_size=num_gpu, # number of gpus
    tensor_parallel_size=1,
    gpu_memory_utilization=0.7
)

turn_end_token_id = 61876 # <|im_end|>
params = SamplingParams(
    temperature=0.01,
    top_p=0.01,
    max_tokens=4096,
    repetition_penalty=1.1,
    stop_token_ids=[turn_end_token_id]
)

def _inference(prompt, llm, params):
    return llm.generate(prompt, params)[0].outputs[0].text


config.json:   0%|          | 0.00/650 [00:00<?, ?B/s]

INFO 09-27 15:50:08 llm_engine.py:161] Initializing an LLM engine (v0.4.3) with config: model='MediaTek-Research/Breeze-7B-FC-v1_0', speculative_config=None, tokenizer='MediaTek-Research/Breeze-7B-FC-v1_0', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=MediaTek-Research/Breeze-7B-FC-v1_0)


tokenizer_config.json:   0%|          | 0.00/2.41k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/911k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/197 [00:00<?, ?B/s]

...

pytorch_model-00003-of-00004.bin:   0%|          | 0.00/4.60G [00:00<?, ?B/s]

pytorch_model-00004-of-00004.bin:   0%|          | 0.00/508M [00:00<?, ?B/s]

pytorch_model-00001-of-00004.bin:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

pytorch_model-00002-of-00004.bin:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

INFO 09-27 15:51:18 model_runner.py:146] Loading model weights took 13.9536 GB
INFO 09-27 15:51:20 gpu_executor.py:83] # GPU blocks: 4856, # CPU blocks: 2048
INFO 09-27 15:51:23 model_runner.py:854] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 09-27 15:51:23 model_runner.py:858] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 09-27 15:51:29 model_runner.py:924] Graph capturing finished in 6 secs.


### Instruction following

In [6]:
from mtkresearch.llm.prompt import MRPromptV2

sys_prompt = ('You are a helpful AI assistant built by MediaTek Research. '
  'The user you are helping speaks Traditional Chinese and comes from Taiwan.')

prompt_engine = MRPromptV2()

conversations = [
    {"role": "system", "content": sys_prompt},
    {"role": "user", "content": "請問什麼是深度學習？"},
]

prompt = prompt_engine.get_prompt(conversations)


output_str = _inference(prompt, llm, params)
result = prompt_engine.parse_generated_str(output_str)

print(result)
# {'role': 'assistant',
#  'content': '深度學習（Deep Learning）是一種機器學習方法，它模仿人類大腦的神經網路結構來
#              處理複雜的數據和任務。在深度學習中，模型由多層人工神經元組成，每個神經元之間有
#              權重連接，並通過非線性轉換進行計算。這些層與層之間的相互作用使模型能夠學習複雜
#              的函數關係或模式，從而解決各種問題，如圖像識別、自然語言理解、語音辨識等。深度
#              學習通常需要大量的數據和強大的計算能力，因此經常使用圖形處理器（GPU）或特殊的
#              加速器來執行。'}


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.93s/it, Generation Speed: 65.18 toks/s]

{'role': 'assistant', 'content': '深度學習（Deep Learning）是一種機器學習方法，它模仿人類大腦的神經網路結構來處理複雜的數據和任務。在深度學習中，模型由多層人工神經元組成，每個神經元之間有權重連接，並通過非線性轉換進行計算。這些層與層之間的相互作用使模型能夠學習複雜的函數關係或模式，從而解決各種問題，如圖像識別、自然語言理解、語音辨識等。深度學習通常需要大量的數據和強大的計算能力，因此經常使用圖形處理器（GPU）或特殊的加速器來執行。'}


### Function Calling

In [7]:
import json

from mtkresearch.llm.prompt import MRPromptV2

functions = [
    {
      "name": "get_current_weather",
      "description": "Get the current weather in a given location",
      "parameters": {
        "type": "object",
        "properties": {
          "location": {
            "type": "string",
            "description": "The city and state, e.g. San Francisco, CA"
          },
          "unit": {
            "type": "string",
            "enum": ["celsius", "fahrenheit"]
          }
        },
        "required": ["location"]
      }
    }
]

def fake_get_current_weather(location, unit=None):
    return {'temperature': 30}

mapping = {
    'get_current_weather': fake_get_current_weather
}

prompt_engine = MRPromptV2()

# stage 1: query
conversations = [
    {"role": "user", "content": "請問台北目前溫度是攝氏幾度？"},
]

prompt = prompt_engine.get_prompt(conversations, functions=functions)

output_str = _inference(prompt, llm, params)
result = prompt_engine.parse_generated_str(output_str)

print(result)
# {'role': 'assistant',
#  'tool_calls': [
#    {'id': 'call_U9bYCBRAbF639uUqfwehwSbw', 'type': 'function',
#     'function': {'name': 'get_current_weather', 'arguments': '{"location": "台北, 台灣", "unit": "celsius"}'}}]}

# stage 2: execute called functions
conversations.append(result)

tool_call = result['tool_calls'][0]
func_name = tool_call['function']['name']
func = mapping[func_name]
arguments = json.loads(tool_call['function']['arguments'])
called_result = func(**arguments)

# stage 3: put executed results
conversations.append(
    {
        'role': 'tool',
        'tool_call_id': tool_call['id'],
        'name': func_name,
        'content': json.dumps(called_result)
    }
)

prompt = prompt_engine.get_prompt(conversations, functions=functions)

output_str2 = _inference(prompt, llm, params)
result2 = prompt_engine.parse_generated_str(output_str2)
print(result2)
# {'role': 'assistant', 'content': '台北目前的溫度是攝氏30度'}


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.77it/s, Generation Speed: 65.51 toks/s]


{'role': 'assistant', 'tool_calls': [{'id': 'call_YCBRAbF639uUqfwehwSbwwla', 'type': 'function', 'function': {'name': 'get_current_weather', 'arguments': '{"location": "台北, 台灣", "unit": "攝氏"}'}}]}


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  4.90it/s, Generation Speed: 64.09 toks/s]

{'role': 'assistant', 'content': '台北目前的溫度是攝氏30度。'}
